In [1]:
import numpy as np
import pandas as pd
from ast import literal_eval

import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel, Phrases

import spacy
from nltk.corpus import stopwords

import pyLDAvis
import pyLDAvis.gensim
pyLDAvis.enable_notebook()

from pandarallel import pandarallel
pandarallel.initialize(progress_bar=True, nb_workers=16)

# import warnings
# warnings.filterwarnings(action='once')

INFO: Pandarallel will run on 16 workers.
INFO: Pandarallel will use Memory file system to transfer data between the main process and workers.


#### Стоит удалить слова-цифры, которые я добавлял вместо обычных numeric токенов. Для этого скопирую список всех слов, которые были вставлены в датасет вместо цифр

In [2]:
ONES_FEMININE = {
    1: ('одна',),2: ('две',),3: ('три',),
    4: ('четыре',),5: ('пять',),6: ('шесть',),
    7: ('семь',),8: ('восемь',),9: ('девять',),
}

ONES = {0: ('ноль',),
    1: ('один',),2: ('два',),3: ('три',),
    4: ('четыре',),5: ('пять',),6: ('шесть',),
    7: ('семь',),8: ('восемь',),9: ('девять',),
}

TENS = {
    0: ('десять',),1: ('одиннадцать',),2: ('двенадцать',),
    3: ('тринадцать',),4: ('четырнадцать',),5: ('пятнадцать',),
    6: ('шестнадцать',),7: ('семнадцать',),8: ('восемнадцать',),
    9: ('девятнадцать',),
}

TWENTIES = {
    2: ('двадцать',),3: ('тридцать',),4: ('сорок',),
    5: ('пятьдесят',),6: ('шестьдесят',),7: ('семьдесят',),
    8: ('восемьдесят',),9: ('девяносто',),
}

HUNDREDS = {
    1: ('сто',),2: ('двести',), 3: ('триста',),
    4: ('четыреста',),5: ('пятьсот',),6: ('шестьсот',),
    7: ('семьсот',),8: ('восемьсот',),9: ('девятьсот',),
}

THOUSANDS = {
    1: ('тысяча', 'тысячи', 'тысяч'),  # 10^3
    2: ('миллион', 'миллиона', 'миллионов'),  # 10^6
    3: ('миллиард', 'миллиарда', 'миллиардов'),  # 10^9
    4: ('триллион', 'триллиона', 'триллионов'),  # 10^12
    5: ('квадриллион', 'квадриллиона', 'квадриллионов'),  # 10^15
    6: ('квинтиллион', 'квинтиллиона', 'квинтиллионов'),  # 10^18
    7: ('секстиллион', 'секстиллиона', 'секстиллионов'),  # 10^21
    8: ('септиллион', 'септиллиона', 'септиллионов'),  # 10^24
    9: ('октиллион', 'октиллиона', 'октиллионов'),  # 10^27
    10: ('нониллион', 'нониллиона', 'нониллионов'),  # 10^30
}

Несколько базовых функция для процессинга текста:

In [23]:
# удаляем числовые токены (иначе слова "тысяча", "девятсот" и подобные будут занимать весь топ самых важных слов)

combined = [ONES, ONES_FEMININE, TENS, TWENTIES, HUNDREDS, THOUSANDS]

numeric = []
for dic in combined:
    for tup in dic.values():
        for string in tup:
            numeric.append(string)
            
def remove_numeric(text):
    return [token for token in text if token not in numeric]

def gen_words(text):
    text = gensim.utils.simple_preprocess(text, deacc=True)
    return remove_numeric(text)

In [ ]:
# Удаляем стоп слова, и мусор короче 2 букв, который в стоп слова не попал

stopwords = stopwords.words('russian')
stopwords.append('это')
stopwords.append('год')

def clear_stopwords(text):
    return [word for word in text if word not in stopwords and len(word) > 2]

#### Baseline model:

In [72]:
df = pd.read_csv('/home/sergey/Python_projects/RU_NER/Project_Data/Data/train_data.csv')

In [73]:
df.drop('augmented_tokens', axis=1, inplace=True)
df.drop('augmented_ner_tags', axis=1, inplace=True)

In [6]:
nlp = spacy.load("ru_core_news_lg", disable=['parser', 'ner'])

In [75]:
df.head(3)

,sentence
0,последний был разбит при сауле жемайтами и зем...
1,научным руководителем был в и арнольд
2,но как же нынче выдают замуж княгиня ни от ког...


In [76]:
def lemmatization(text):
    doc = nlp(text)
    new_text = []
    for token in doc:
        new_text.append(token.lemma_)
    return " ".join(new_text)
        

In [77]:
df['spacy'] = df['sentence'].parallel_apply(lambda x: lemmatization(x))

In [79]:
df['gensim_words'] = df['spacy'].parallel_apply(lambda x: gen_words(x))

In [80]:
df.gensim_words = df.gensim_words.parallel_apply(lambda x: remove_numeric(x))

In [81]:
dictionary = corpora.Dictionary(df['gensim_words'])
corpus = [dictionary.doc2bow(doc) for doc in df['gensim_words']]

In [82]:
lda_model_baseline = gensim.models.ldamulticore.LdaMulticore(
    workers=16,
    corpus = corpus,
    id2word = dictionary,
    num_topics = 15,
    random_state = 13,
    eval_every = 2,
    chunksize = 2000,
    passes = 10,
)

In [83]:
def compute_coherence(model, texts, id2word, coherence='c_v'):
    coherence_model_lda = CoherenceModel(model=model,
                                         texts=texts,
                                         dictionary=id2word,
                                         coherence=coherence)
    coherence_lda = coherence_model_lda.get_coherence()
    print('Coherence Score: ', coherence_lda)

In [84]:
compute_coherence(lda_model_baseline, df['gensim_words'], dictionary)

Coherence Score:  0.3779725303117619


In [85]:
vis=pyLDAvis.gensim.prepare(lda_model_baseline, corpus, dictionary, mds='mmds', R=15)

In [86]:
pyLDAvis.save_html(vis, 'LDAvis/baseline.html')

In [87]:
lda_model_baseline.print_topics()

[(0,
  '0.015*"он" + 0.013*"на" + 0.010*"год" + 0.008*"для" + 0.006*"работать" + 0.005*"что" + 0.005*"время" + 0.005*"быть" + 0.005*"министр" + 0.005*"союз"'),
 (1,
  '0.023*"год" + 0.020*"город" + 0.016*"по" + 0.013*"на" + 0.008*"центр" + 0.007*"население" + 0.007*"входить" + 0.006*"он" + 0.006*"сша" + 0.006*"после"'),
 (2,
  '0.016*"на" + 0.013*"год" + 0.010*"для" + 0.008*"были" + 0.007*"он" + 0.006*"быть" + 0.006*"конец" + 0.006*"век" + 0.006*"индия" + 0.006*"из"'),
 (3,
  '0.037*"не" + 0.026*"что" + 0.016*"это" + 0.013*"этот" + 0.011*"мы" + 0.010*"на" + 0.010*"он" + 0.010*"быть" + 0.009*"но" + 0.008*"весь"'),
 (4,
  '0.018*"на" + 0.018*"вид" + 0.015*"из" + 0.013*"род" + 0.011*"как" + 0.010*"семеиство" + 0.010*"или" + 0.007*"растение" + 0.007*"также" + 0.006*"что"'),
 (5,
  '0.025*"город" + 0.024*"остров" + 0.018*"на" + 0.014*"штат" + 0.013*"раион" + 0.012*"река" + 0.012*"километр" + 0.009*"область" + 0.009*"из" + 0.009*"центр"'),
 (6,
  '0.043*"год" + 0.016*"быть" + 0.014*"на" + 0.

Функция для экспериментов с параметрами:

In [58]:
def train_compute_coherence_save_chart(name, corpus, dictionary, k, a, b):
    """
    1) Train Model
    2) Compute Coherence score
    3) Create and save visualization
    """
    
    lda_model = gensim.models.ldamulticore.LdaMulticore(workers=16,
                                                        corpus = corpus,
                                                        id2word = dictionary,
                                                        num_topics = k,
                                                        random_state = 13,
                                                        eval_every = 2,
                                                        chunksize = 2000,
                                                        alpha=a,
                                                        eta=b,
                                                        passes = 20)
    
    coherence_model_lda = CoherenceModel(model=lda_model,
                                         texts=df['gensim_words'],
                                         dictionary=dictionary,
                                         coherence='c_v')
    
    vis=pyLDAvis.gensim.prepare(lda_model, corpus, dictionary, mds='mmds', R=k)
    pyLDAvis.save_html(vis, f'LDAvis/{name}.html')
    print(coherence_model_lda.get_coherence())
    return lda_model, coherence_model_lda.get_coherence()

___
#### Adjusting lemmatization

In [90]:
df = pd.read_csv('/home/sergey/Python_projects/RU_NER/Project_Data/Data/train_data.csv')

In [91]:
df.drop('augmented_tokens', axis=1, inplace=True)
df.drop('augmented_ner_tags', axis=1, inplace=True)

In [24]:
# Попробуем оставить только определенные части речи

def lemmatization(text, allowed_postages=['NOUN', 'ADJ', 'VERB', 'ADV']):
    doc = nlp(text)
    new_text = []
    for token in doc:
        if token.pos_ in allowed_postages:
            new_text.append(token.lemma_)
    return " ".join(new_text)

In [93]:
df['spacy'] = df['sentence'].parallel_apply(lambda x: lemmatization(x))

In [94]:
df['gensim_words'] = df['spacy'].parallel_apply(lambda x: gen_words(x))

In [95]:
# df.to_csv('lemmatized_for_topics.csv', index=False)

In [37]:
# df['gensim_words'] = df['gensim_words'].parallel_apply(literal_eval)

KeyError: 'gensim_words'

In [96]:
dictionary = corpora.Dictionary(df['gensim_words'])
corpus = [dictionary.doc2bow(doc) for doc in df['gensim_words']]

In [97]:
adjusted_lemmatization_coherence = train_compute_coherence_save_chart('adjusted_lemm',
                                                                      corpus, dictionary, 15, 1, 1)

0.35893649478202744


In [98]:
adjusted_lemmatization_coherence 

0.35893649478202744

___
#### Bigrams + Trigrams and stopwords removal


In [8]:
df = pd.read_csv('/home/sergey/Python_projects/RU_NER/Project_Data/Data/train_data.csv')

In [9]:
df.drop('augmented_tokens', axis=1, inplace=True)
df.drop('augmented_ner_tags', axis=1, inplace=True)

In [10]:
def lemmatization(text, allowed_postages=['NOUN', 'ADJ', 'VERB', 'ADV']):
    doc = nlp(text)
    new_text = []
    for token in doc:
        if token.pos_ in allowed_postages:
            new_text.append(token.lemma_)
    return " ".join(new_text)

In [11]:
#  Пробовал использовать стандартную лемматизцию, результат был хуже
#def lemmatization(text):
#     doc = nlp(text)
#     new_text = []
#     for token in doc:
#         new_text.append(token.lemma_)
#     return " ".join(new_text)

In [12]:
df['spacy'] = df['sentence'].parallel_apply(lambda x: lemmatization(x))

In [17]:
df['gensim_words'] = df['spacy'].parallel_apply(lambda x: gen_words(x))

In [25]:
bigram = gensim.models.Phrases(df['gensim_words'], min_count = 5, threshold=50)
trigram = gensim.models.Phrases(bigram[df['gensim_words']], threshold=50)

bigram_mod = gensim.models.phrases.Phraser(bigram)
trigram_mod = gensim.models.phrases.Phraser(trigram)

In [26]:
df['gensim_words'] = df['gensim_words'].parallel_apply(lambda x: clear_stopwords(x))

In [27]:
def make_bi_trigrams(text):
    return trigram_mod[bigram_mod[text]]

In [28]:
df['gensim_words'] = df['gensim_words'].parallel_apply(lambda x: make_bi_trigrams(x))

In [113]:
# df.to_csv('ngrams.csv', index=False)

In [4]:
df = pd.read_csv('ngrams.csv')

In [6]:
df['gensim_words'] = df['gensim_words'].parallel_apply(literal_eval)

In [29]:
dictionary = corpora.Dictionary(df['gensim_words'])
corpus = [dictionary.doc2bow(doc) for doc in df['gensim_words']]

In [40]:
bi_and_trigras_coherence = train_compute_coherence_save_chart('ngrams_lemm',
                                                                      corpus, dictionary, 15, 0.01, 0.01)

0.34917137976291585


In [44]:
#with filter:
dictionary = corpora.Dictionary(df['gensim_words'])
dictionary.filter_extremes(no_below=30, no_above=0.5)
corpus = [dictionary.doc2bow(doc) for doc in df['gensim_words']]

In [42]:
len(dictionary)

4018

In [43]:
filter_extremes_coherence = train_compute_coherence_save_chart('filter_extremes_lemm',
                                                                      corpus, dictionary, 15, 0.01, 0.01)

0.3693857745486614


In [30]:
import tqdm

In [35]:
min_topics = 5
max_topics = 30
step_size = 5
topics_range = range(min_topics, max_topics, step_size)

In [36]:
alpha = list(np.arange(0.01, 1, 0.25))
alpha.append('symmetric')
alpha.append('asymmetric')

In [37]:
beta = list(np.arange(0.01, 1, 0.25))
beta.append('symmetric')

In [95]:
model_results = {'Topics': [],
                 'Alpha': [],
                 'Beta': [],
                 'Coherence': []
                }

In [103]:
def train_compute_coherence_save_chart(corpus, dictionary, k, a, b):
    """
    1) Train Model
    2) Compute Coherence score
    """
    
    lda_model = gensim.models.ldamulticore.LdaMulticore(workers=16,
                                                        corpus = corpus,
                                                        id2word = dictionary,
                                                        num_topics = k,
                                                        random_state = 13,
                                                        eval_every = 2,
                                                        chunksize = 2000,
                                                        alpha=a,
                                                        eta=b,
                                                        passes = 10)
    
    coherence_model_lda = CoherenceModel(model=lda_model,
                                         texts=df['gensim_words'],
                                         dictionary=dictionary,
                                         coherence='c_v')
    
#     vis=pyLDAvis.gensim.prepare(lda_model, corpus, dictionary, mds='mmds', R=k)
#     pyLDAvis.save_html(vis, f'LDAvis/{name}.html')
    print(coherence_model_lda.get_coherence())
    return coherence_model_lda.get_coherence()

In [104]:
pbar.close()
if 1 == 1:
    pbar = tqdm.tqdm(total=(len(beta)*len(alpha)*len(topics_range)))
    
    for k in topics_range:
        for a in alpha:
            for b in beta:
                cv = train_compute_coherence_save_chart(corpus=corpus, dictionary=dictionary, 
                                              k=k, a=a, b=b)
                model_results['Topics'].append(k)
                model_results['Alpha'].append(a)
                model_results['Beta'].append(b)
                model_results['Coherence'].append(cv)

                pbar.update(1)
    pd.DataFrame(model_results).to_csv('./results/lda_tuning_results.csv', index=False)
    pbar.close()


  1%|▎                                        | 1/150 [00:44<1:49:23, 44.05s/it]

0.3284127530364974


  1%|▌                                        | 2/150 [01:27<1:47:44, 43.68s/it]

0.3311698588279435


  2%|▊                                        | 3/150 [02:10<1:46:50, 43.61s/it]

...

0.3693399780324968


 99%|██████████████████████████████████████▋| 149/150 [1:49:50<00:41, 41.65s/it]

0.34528868158314224


100%|███████████████████████████████████████| 150/150 [1:50:30<00:00, 41.20s/it]

OSError: Cannot save file into a non-existent directory: 'results'

In [106]:
df_res = pd.DataFrame(model_results)

In [33]:
df_res.to_csv('lda_tuning_results.csv', index=False)

In [31]:
df_res = pd.read_csv('lda_tuning_results.csv')

In [34]:
df_res.sort_values(by='Coherence')

,Topics,Alpha,Beta,Coherence
14,5,0.51,0.01,0.248649
44,10,0.51,0.01,0.249417
19,5,0.76,0.01,0.250682
49,10,0.76,0.01,0.250773
3,5,0.01,0.01,0.253108
...,...,...,...,...
111,20,0.76,0.51,0.398717
36,10,0.01,0.51,0.408103
112,20,0.76,0.76,0.413216
141,25,0.76,0.51,0.424364


Я просмотрел несколько лучших вариантов по параметрам, и чсто визуально мне больше вссего понравился такйо вариант:

In [59]:
lda_model, cv = train_compute_coherence_save_chart(name='best_model_vis', corpus=corpus, dictionary=dictionary, 
                                              k=15, a=0.51, b=0.51)

0.3848752897496348


In [70]:
lda_model.save('best_model.model')